In [102]:
%reload_ext autoreload
%autoreload 2


from bartez.boards import *
#get_board_preset_small_01 
blocks, geometry = generate_board_from_string(get_board_preset_01())

In [103]:
from bartez.crossword import Crossworld

crossword = Crossworld(geometry[0], geometry[1])
crossword.set_blocks(blocks)
crossword.prepare()
crossword.print_crossword()
#crossword.print_entries()


. . . . # . . . . . . . # . . . . . 
. . . . . . . . . # . # . . . . . . 
. . # . . . . . # . . . . . . . # . 
. # . . . . . # . . . . . . . # . . 
# . . . . . # . . . . . . . # . . . 
. . . . . # . . . . . . . # . . . . 
. . . . # . . . . . . . # . . . . . 
. . . # . . . . . . . # . . . . . # 
. . # . . . . . . . # . . . . . # . 
. # . . . . . . . # . . . . . # . . 
. . . . . . # . # . . . . . . . . . 
. . . . . # . . . . . . . # . . . . 



In [146]:
import networkx as nx

G = nx.Graph()
entries = crossword.get_entries()
nodes_h = []
nodes_v = []
for i, e in enumerate(entries):
    #print(e.get_description() + ", index: " + str(e.absolute_index()))
    relations = e.get_relations()
    is_horizontal = e.is_horizontal()
    e_index = e.get_absolute_index()
    for rel in relations:
        G.add_edge(e_index, rel.get_index())
    
    G.nodes[i]['id']=e_index
    G.nodes[i]['title']=e.get_description()
    G.nodes[i]['horizontal']=is_horizontal
    G.nodes[i]['color']= "00ff1e" if is_horizontal else "0bcde"
    
    if is_horizontal:
        nodes_h.append(e_index)
    else:
        nodes_v.append(e_index)

#print(G.nodes())
#print(G.edges())

In [147]:
%matplotlib notebook

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt


In [148]:
from pyvis.network import Network
import networkx as nx

nx_graph = nx.cycle_graph(10)
nx_graph.nodes[1]['title'] = 'Number 1'
nx_graph.nodes[1]['group'] = 1
nx_graph.nodes[3]['title'] = 'I belong to a different group!'
nx_graph.nodes[3]['group'] = 10
nx_graph.add_node(20, size=20, title='couple', group=2)
nx_graph.add_node(21, size=15, title='couple', group=2)
nx_graph.add_edge(20, 21, weight=5)
nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)
nx_graph = G
nt = Network('800px', '800px', notebook=True)
# populates the nodes and edges data structures
nt.from_nx(nx_graph)
nt.show('nx.html')

In [138]:
crossword.print_entries()

0: 1 Horizontal [....] r,c: [0, 0] -> [0, 3]
1: 1 Vertical   [....] r,c: [0, 0] -> [3, 0]
2: 2 Vertical   [...] r,c: [0, 1] -> [2, 1]
3: 3 Vertical   [..] r,c: [0, 2] -> [1, 2]
4: 4 Vertical   [.......] r,c: [0, 3] -> [6, 3]
5: 5 Horizontal [.......] r,c: [0, 5] -> [0, 11]
6: 5 Vertical   [.....] r,c: [0, 5] -> [4, 5]
7: 6 Vertical   [....] r,c: [0, 6] -> [3, 6]
8: 7 Vertical   [...] r,c: [0, 7] -> [2, 7]
9: 8 Vertical   [..] r,c: [0, 8] -> [1, 8]
10: 9 Vertical   [........] r,c: [0, 10] -> [7, 10]
11: 10 Horizontal [.....] r,c: [0, 13] -> [0, 17]
12: 10 Vertical   [.....] r,c: [0, 13] -> [4, 13]
13: 11 Vertical   [....] r,c: [0, 14] -> [3, 14]
14: 12 Vertical   [...] r,c: [0, 15] -> [2, 15]
15: 13 Vertical   [..] r,c: [0, 16] -> [1, 16]
16: 14 Vertical   [.......] r,c: [0, 17] -> [6, 17]
17: 15 Horizontal [.........] r,c: [1, 0] -> [1, 8]
18: 16 Vertical   [.....] r,c: [1, 4] -> [5, 4]
19: 17 Horizontal [......] r,c: [1, 12] -> [1, 17]
20: 17 Vertical   [.....] r,c: [1, 12] -> [5, 12]